In [1]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

from rpy2.robjects import numpy2ri
from rpy2.robjects import pandas2ri
from rpy2.robjects import r

import numpy as np
import pandas as pd

/home/harveyaa/miniconda3/envs/GPyTorch/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
numpy2ri.activate()
pandas2ri.activate()

In [3]:
utils = importr('utils')
utils.chooseCRANmirror(ind=1)
utils.install_packages('gamlss')
gamlss_data = importr('gamlss.data',lib_loc='/home/harveyaa/R/x86_64-pc-linux-gnu-library/3.6/')
gamlss_dist = importr('gamlss.dist',lib_loc='/home/harveyaa/R/x86_64-pc-linux-gnu-library/3.6/')
gamlss = importr('gamlss',lib_loc='/home/harveyaa/R/x86_64-pc-linux-gnu-library/3.6/')

R[write to console]: Installing package into ‘/home/harveyaa/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/gamlss_5.3-4.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 325899 bytes (318 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

In [4]:
def sample_x(low=1,high=100,n_subs=1000,sampling='full'):
    if sampling =='full':
        x = np.random.uniform(low=low,high=high,size=n_subs)
    else:
        x = np.concatenate([np.random.normal(20,10,size=int(n_subs/2)),np.random.normal(80,10,size=int(n_subs/2))])
        x = x[(x<high) & (x > low)]
    return x

# Heteroskedastic linear
def dataset(low=1,high=100,n_subs=1000,sampling='full'):
    x = sample_x(low=low,high=high,n_subs=n_subs,sampling=sampling)
    scores = np.array([np.log(i) + 0.15*np.log(i)*np.random.randn() for i in x])
    df = pd.DataFrame([x,scores],index=['x','score']).transpose()
    return df

In [5]:
df = dataset()

In [6]:
NO = r['NO']
formula = r['formula']
f = formula('score~poly(x,3)')
g = gamlss.gamlss(f, data=df,family=NO())

GAMLSS-RS iteration 1: Global Deviance = 1725.415 
GAMLSS-RS iteration 2: Global Deviance = 1725.415 


In [7]:
summary = r['summary']
summary(g)

02565467147899, 3.67046030792808,  
    1.78737678024365, 4.00123773578848, 3.4485467562977, 3.83686731261243,  
    3.65379248506352, 2.03786363333414, 3.71129490662027, 2.30590927078595,  
    4.31840879482997, 3.92488101517371, 3.26345544337068, 3.40718562004877,  
    3.10156734895332, 5.13720755212401, 2.76783560223938, 3.75072778004961,  
    1.86410828739432, 3.19925625889641, 5.0081582975809, 2.4722580689772,  
    4.57088999285799, 4.17295019262019, 4.2907830341602, 3.25099225367166,  
    5.71343805336816, 4.48081961424511, 4.74195424629074, 3.25517806207481,  
    4.34985697423343, 3.65361333280178, 5.26083018775636, 5.76078019913102,  
    4.54582429527, 2.18008237693556, 3.44945000897027, 4.82100907194869,  
    1.10902813482389, 3.54445261620924, 2.96802831387384, 3.36616118177265,  
    1.61777279340721, 4.63245849751036, 3.81057766743657, 3.98287325602164,  
    4.21516915192128, 3.99980520106832, 3.70531650296613, 4.20383851852361,  
    2.87036261610052, 3.26447691555

array([[ 3.59969540e+000,  1.81314285e-002,  1.98533469e+002,
         0.00000000e+000],
       [ 2.57287476e+001,  5.73366112e-001,  4.48731571e+001,
         2.11600519e-241],
       [-9.89575341e+000,  5.73366112e-001, -1.72590483e+001,
         1.35208522e-058],
       [ 6.23502947e+000,  5.73366112e-001,  1.08744297e+001,
         4.18181634e-026],
       [-5.56230828e-001,  2.23606798e-002, -2.48753988e+001,
         1.08892576e-106]])

In [59]:
g[13]

array([1], dtype=int32)

In [61]:
cs = r['cs']
f2 = formula('score~cs(x,df=3)')
g2 = gamlss.gamlss(f2,data=df,family=NO())

GAMLSS-RS iteration 1: Global Deviance = 1770.067 
GAMLSS-RS iteration 2: Global Deviance = 1770.067 


In [63]:
predict = r['predict']
predict(g2,what='sigma',type='response')[1]

0.5863109588633391

In [64]:
fitted = r['fitted']
fitted(g2,"sigma")[1]

0.5863109588633391

In [68]:
g3 = gamlss.gamlss(f2,sigma_formula=formula('~cs(x,3)'),data=df,family=NO())

GAMLSS-RS iteration 1: Global Deviance = 1708.082 
GAMLSS-RS iteration 2: Global Deviance = 1699.646 
GAMLSS-RS iteration 3: Global Deviance = 1698.663 
GAMLSS-RS iteration 4: Global Deviance = 1698.44 
GAMLSS-RS iteration 5: Global Deviance = 1698.369 
GAMLSS-RS iteration 6: Global Deviance = 1698.347 
GAMLSS-RS iteration 7: Global Deviance = 1698.339 
GAMLSS-RS iteration 8: Global Deviance = 1698.337 
GAMLSS-RS iteration 9: Global Deviance = 1698.335 
GAMLSS-RS iteration 10: Global Deviance = 1698.335 


In [70]:
resid = r['residuals']
resid(g3)

array([-2.38704391e+00, -1.20444985e-01,  2.77362893e-01,  9.19865731e-02,
        9.76881368e-01,  8.63147812e-01,  8.40354693e-01,  9.86139611e-01,
        6.80144323e-01, -1.80452297e+00,  2.28242809e-02,  1.41148279e+00,
        1.41712566e-01, -4.03387622e-01,  1.53023265e+00,  1.57820853e+00,
        1.02675578e+00, -5.04893617e-01,  6.69146432e-01,  9.38183780e-01,
       -8.47353408e-01, -6.80962776e-01, -1.28056403e+00,  7.15260800e-01,
       -3.90037914e-01, -1.65556282e-01, -1.98164458e+00,  1.38765401e-01,
       -4.44314111e-01, -6.96634388e-01, -6.00754726e-01,  2.38821198e+00,
       -1.03212532e+00,  3.17188011e-01, -2.79026821e-01,  5.34448467e-01,
       -1.58012341e+00,  6.69511463e-01, -2.13453005e-01,  1.99097848e+00,
       -2.95223910e-01, -3.36964886e-01, -6.83964516e-01,  8.89793270e-01,
       -1.02000863e+00, -5.76303299e-01,  1.72770831e+00,  7.17115406e-01,
        2.01490699e-01,  4.28974527e-01,  2.11455285e+00, -4.27558079e-01,
       -3.28042101e-01, -

In [74]:
plot = r['plot']
plot(g3)

******************************************************************
	      Summary of the Quantile Residuals
                           mean   =  -0.003772006 
                       variance   =  1.000987 
               coef. of skewness  =  0.01337103 
               coef. of kurtosis  =  2.977382 
Filliben correlation coefficient  =  0.9993427 
******************************************************************


mfrow,[RTYPES.INTSXP]
mar,[RTYPES.REALSXP]
col.axis,[RTYPES.STRSXP]
col.main,[RTYPES.STRSXP]
col.lab,[RTYPES.STRSXP]
col,[RTYPES.STRSXP]
bg,[RTYPES.STRSXP]


In [9]:
SHASHo2 = r['SHASHo2']
g4 = gamlss.gamlss(formula('score~lo(~x,span=.4)'),
                    sigma_formula=formula('~lo(~x,span=.4)'),
                    data=df,
                    family=SHASHo2)

GAMLSS-RS iteration 1: Global Deviance = 1896.31 
GAMLSS-RS iteration 2: Global Deviance = 1699.117 
GAMLSS-RS iteration 3: Global Deviance = 1641.989 
GAMLSS-RS iteration 4: Global Deviance = 1620.39 
GAMLSS-RS iteration 5: Global Deviance = 1610.704 
GAMLSS-RS iteration 6: Global Deviance = 1606.056 
GAMLSS-RS iteration 7: Global Deviance = 1603.426 
GAMLSS-RS iteration 8: Global Deviance = 1601.943 
GAMLSS-RS iteration 9: Global Deviance = 1601.043 
GAMLSS-RS iteration 10: Global Deviance = 1600.468 
GAMLSS-RS iteration 11: Global Deviance = 1600.083 
GAMLSS-RS iteration 12: Global Deviance = 1599.868 
GAMLSS-RS iteration 13: Global Deviance = 1599.673 
GAMLSS-RS iteration 14: Global Deviance = 1599.506 
GAMLSS-RS iteration 15: Global Deviance = 1599.395 
GAMLSS-RS iteration 16: Global Deviance = 1599.313 
GAMLSS-RS iteration 17: Global Deviance = 1599.253 
GAMLSS-RS iteration 18: Global Deviance = 1599.209 
GAMLSS-RS iteration 19: Global Deviance = 1599.174 
GAMLSS-RS iteration 20:

In [81]:
AIC = r['AIC']
AIC(g2,g3,g4,k=3)

,df,AIC


In [10]:
r('''
 matcal <- function(mat) {
 s = sum(mat)
 sigma = sd(mat)
 return(c(s,sigma))
 }
''')
## This function can now be accessed with the __getitem__ mechanins
r_matcal = robjects.globalenv['matcal']
print(r_matcal.r_repr())

function (mat) 
{
    s = sum(mat)
    sigma = sd(mat)
    return(c(s, sigma))
}


In [17]:
A = np.random.randn(3,3)
r_matcal(A)

-3.113474,1.006387
